## EXAMPLE OF VIDEO-QA IMPLEMENTATION WITH TENSORFLOW 1.2RC0

EFFECTIVE TENSORFLOW FOR NON-EXPERTS https://www.youtube.com/watch?v=5DknTFbcGVM

In [ ]:
### Turning frames into a vector, with pre-trained representations
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import tensorflow as tf
print("tf.__version__ : ", tf.__version__)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import InceptionV3

video = keras.Input(shape=(None, 150, 150, 3), name='video')
cnn = InceptionV3(weights='imagenet',
                  include_top=False,
                  pooling='avg')
cnn.trainable = False
frame_features = layers.TimeDistributed(cnn)(video)
video_vector = layers.LSTM(256)(frame_features)

### Turning a sequence of words into a vector
question = keras.Input(shape=(None, ), dtype='int32', name='question')
embedded_words = layers.Embedding(input_voc_size, 256)(question)
question_vector = layers.LSTM(128)(embedded_words)

### Predicting an answer word
x = layers.concatenate([video_vector, question_vector])
x = layers.dense(128, activation=tf.nn.relu)(x)
predictions = layers.dense(output_voc_size, name='predictions')(x)

### Setting up the training configuration
model = keras.models.Model([video, question], predictions)
model.compile(optimizer=tf.AdamOptimizer(),
              loss=tf.softmax_crossentropy_with_logits)


### Leveraging Experiment for distributed training
def experiment_fn(config, params):
    model = ...
    estimator = model.get_estimator(config=...)
    return Experiment(estimator,
                      train_input_fn=pandas_input_fn(...),
                      eval_input_fn=pandas_input_fn(...))


if __name__ == '__main__':
    tf.contrib.train.run_experiment(experiment_fn)